### Card and Deck objects

This notebook contains example code from [*Fluent Python*](http://shop.oreilly.com/product/0636920032519.do), by Luciano Ramalho.

Code by Luciano Ramalho, modified by Allen Downey.

MIT License: https://opensource.org/licenses/MIT

This example demonstrates the Python data model using a simple implementation of playing cards and decks.

`Card` is a namedtuple that represents a playing card.

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

`FrenchDeck` is a class that represents a deck of cards.

In [2]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

You can instantiate a `Card` object as if `Card` were a class.

BTW: [beer card](https://en.wikipedia.org/wiki/Beer_card)

In [3]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

You can access the fields of a card by name.

In [4]:
beer_card.rank, beer_card.suit

('7', 'diamonds')

Or by index.

In [5]:
beer_card[0], beer_card[1]

('7', 'diamonds')

A drawback of using namedtuples is that you can't define methods for them in the usual way.

But you can [monkey-patch](https://en.wikipedia.org/wiki/Monkey_patch) them by defining a function and then making it an attribute of `Card`.  For example, here's a function that generates a string representation of a card:

In [6]:
def card_to_str(card):
    return '%s of %s' % card

card_to_str(beer_card)

'7 of diamonds'

Here's how we can make that function behave like a method.  When we pass a card to `print`, Python invokes the special method `__str__`

In [7]:
Card.__str__ = card_to_str
print(beer_card)

7 of diamonds


Now let's instantiate a `FrenchDeck`.

When we call `len`, Python invokes the `__len__` method on the deck. 

In [8]:
deck = FrenchDeck()
len(deck)

52

When we use the bracket operator, Python invokes the `__getitem__` method:

In [9]:
deck[3]

Card(rank='5', suit='spades')

And that means that the slice operator works, too:

In [10]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

Aside: In this context, we don't get the string generated by `__str__`; we get the one generated by `__repr__` (read about that [here](https://docs.python.org/3/reference/datamodel.html#basic-customization)) 

Because `FrenchDeck` provides `__len__` and `__getitem__`, it is considered a sequence, which means that the `in` operator works:

In [11]:
Card('Q', 'hearts') in deck

True

**Exercise** Make up a card that doesn't exist and confirm that `in` returns `False`.

In [12]:
# Solution goes here

And the for loop works, too:

In [13]:
for card in deck:
    print(card)

2 of spades
3 of spades
4 of spades
5 of spades
6 of spades
7 of spades
8 of spades
9 of spades
10 of spades
J of spades
Q of spades
K of spades
A of spades
2 of diamonds
3 of diamonds
4 of diamonds
5 of diamonds
6 of diamonds
7 of diamonds
8 of diamonds
9 of diamonds
10 of diamonds
J of diamonds
Q of diamonds
K of diamonds
A of diamonds
2 of clubs
3 of clubs
4 of clubs
5 of clubs
6 of clubs
7 of clubs
8 of clubs
9 of clubs
10 of clubs
J of clubs
Q of clubs
K of clubs
A of clubs
2 of hearts
3 of hearts
4 of hearts
5 of hearts
6 of hearts
7 of hearts
8 of hearts
9 of hearts
10 of hearts
J of hearts
Q of hearts
K of hearts
A of hearts


Other methods that work with sequences, like `random.choice`, will work with decks:

In [14]:
from random import choice
choice(deck)

Card(rank='3', suit='hearts')

Sadly, `shuffle` doesn't work because we haven't provided `__setitem__`, so a deck is an immutable sequence:

In [15]:
from random import shuffle

# This should raise a TypeError
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

We can use `sorted` to iterate through the cards in the order determined by tuple comparison:

In [ ]:
for card in sorted(deck):
    print(card)

If we want an ordering that makes more sense for cards, we can define a function that maps from a card to an integer:

In [ ]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high_ordering(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [ ]:
spades_high_ordering(Card('2', 'clubs'))

In [ ]:
spades_high_ordering(Card('A', 'spades'))

And then pass this funcition as a key to `sorted`:

In [ ]:
for card in sorted(deck, key=spades_high_ordering):
    print(card)

**Exercise**  Define a new ordering that sorts the cards by suit first and then by rank, so all clubs come first, followed by all diamonds, etc.

In [ ]:
# Solution goes here

**Exercise**  Write a method called `setcard` that takes a deck, an index, and a card, and assigns the card to the deck at the given position.  Then monkey-patch `FrenchDeck` to provide `__setitem__` as a method.  Test it by assigning a new card like this:

    deck[0] = Card('A', 'spades')

Then shuffle the deck using `random.shuffle`.

In [ ]:
# Solution goes here

We should have two Aces of spades now, which we can confirm by checking the number of unique cards:

In [ ]:
len(set(deck))